# Welcome to Full Stack Machine Learning's Week 4 Project!

In the final week, you will return to the workflow you built last week on the [taxi dataset](https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page). 

## Task 1: Deploy the champion
Use what you have learned in the last two weeks to make necessary modifications and to deploy your latest version of the `TaxiFarePrediction` flow to Argo. Use `--branch champion` to denote this deployment as the champion model.

In [17]:
# Having troubles reading the parquet file, eventhough I can access it via Chrome. Reverting to single quotations seems to do the trick... Not sure why.
import pandas as pd
dd = pd.read_parquet('https://outerbounds-datasets.s3.us-west-2.amazonaws.com/taxi/latest.parquet')
dd

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,hour
0,2,2023-05-10 08:32:10,2023-05-10 08:40:36,1.0,0.97,1.0,N,161,141,2,9.30,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00,0
1,2,2023-05-10 08:55:08,2023-05-10 09:01:27,1.0,1.10,1.0,N,43,237,1,7.90,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00,1
2,2,2023-05-10 08:25:04,2023-05-10 08:37:49,1.0,2.51,1.0,N,48,238,1,14.90,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00,0
3,1,2023-05-10 08:03:48,2023-05-10 08:13:25,0.0,1.90,1.0,N,138,7,1,12.10,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25,0
4,2,2023-05-10 08:10:29,2023-05-10 08:21:19,1.0,1.43,1.0,N,107,79,1,11.40,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3018517,1,2023-05-21 09:16:24,2023-05-21 09:36:04,NaN,4.70,NaN,None,239,42,0,21.20,1.00,0.5,5.24,0.0,1.0,31.44,NaN,NaN,265
3018518,2,2023-05-21 09:04:41,2023-05-21 09:31:03,NaN,6.10,NaN,None,186,112,0,23.11,0.00,0.5,5.42,0.0,1.0,32.53,NaN,NaN,265
3018520,2,2023-05-21 09:11:13,2023-05-21 09:18:52,NaN,2.23,NaN,None,114,170,0,15.48,0.00,0.5,3.90,0.0,1.0,23.38,NaN,NaN,265
3018521,1,2023-05-21 09:40:59,2023-05-21 09:54:11,NaN,0.00,NaN,None,141,42,0,19.54,0.00,0.5,0.00,0.0,1.0,23.54,NaN,NaN,265


In [43]:
%%writefile ../flows/cloud/foo_taxi_fare_champion_1.py

# Let's use a modified version of the event_triggered_linear_regression.py file from week 3 as the champion.
# Note that I encountered problems with reading the parquet url, and potentially saw some issues with caching - I had to explicitly state the parquet url and make sure
# to write the contents of the cell to a new file, using a different flow class name to make it work. Did not get to the root cause of the problem due to time limitations.

from metaflow import FlowSpec, step, card, conda_base, current, Parameter, Flow, trigger, project
from metaflow.cards import Markdown, Table, Image, Artifact


URL = 'https://outerbounds-datasets.s3.us-west-2.amazonaws.com/taxi/latest.parquet'
DATETIME_FORMAT = '%Y-%m-%d %H:%M:%S'

@project(name="foo_taxi_fare")
@trigger(events=['s3'])
@conda_base(libraries={'pandas': '1.4.2', 'pyarrow': '11.0.0', 'numpy': '1.21.2', 'scikit-learn': '1.1.2'})
class TaxiFarePrediction_Foo(FlowSpec):

    data_url = Parameter("data_url", default='https://outerbounds-datasets.s3.us-west-2.amazonaws.com/taxi/latest.parquet')

    def transform_features(self, df):

        # TODO: 
            # Try to complete tasks 2 and 3 with this function doing nothing like it currently is.
            # Understand what is happening.
            # Revisit task 1 and think about what might go in this function.
        obviously_bad_data_filters = [

            df.fare_amount > 0,         # fare_amount in US Dollars
            df.trip_distance <= 100,    # trip_distance in miles
            df.trip_distance > 0,
            df.passenger_count > 0,
            df.extra > 0,
            df.mta_tax > 0,
            df.tip_amount >= 0,
            df.tolls_amount >= 0,

            # TODO: add some logic to filter out what you decide is bad data!
            # TIP: Don't spend too much time on this step for this project though, it practice it is a never-ending process.

        ]

        for f in obviously_bad_data_filters:
            df = df[f]
        return df

    @step
    def start(self):

        import pandas as pd
        import io
        import requests
        from sklearn.model_selection import train_test_split

        self.df = self.transform_features(pd.read_parquet('https://outerbounds-datasets.s3.us-west-2.amazonaws.com/taxi/latest.parquet'))

        # NOTE: we are split into training and validation set in the validation step which uses cross_val_score.
        # This is a simple/naive way to do this, and is meant to keep this example simple, to focus learning on deploying Metaflow flows.
        # In practice, you want split time series data in more sophisticated ways and run backtests. 
        self.X = self.df["trip_distance"].values.reshape(-1, 1)
        self.y = self.df["total_amount"].values
        self.next(self.linear_model)

    @step
    def linear_model(self):
        "Fit a single variable, linear model to the data."
        from sklearn.linear_model import LinearRegression

        # TODO: Play around with the model if you are feeling it.
        self.model = LinearRegression()
        self.model_type = "linear regressor"
        self.next(self.validate)

    def gather_sibling_flow_run_results(self):

        # storage to populate and feed to a Table in a Metaflow card
        rows = []

        # loop through runs of this flow 
        for run in Flow(self.__class__.__name__):
            if run.id != current.run_id:
                if run.successful:
                    icon = "✅" 
                    msg = "OK"
                    score = str(run.data.scores.mean())
                else:
                    icon = "❌"
                    msg = "Error"
                    score = "NA"
                    for step in run:
                        for task in step:
                            if not task.successful:
                                msg = task.stderr
                row = [Markdown(icon), Artifact(run.id), Artifact(run.created_at.strftime(DATETIME_FORMAT)), Artifact(score), Markdown(msg)]
                rows.append(row)
            else:
                rows.append([Markdown("✅"), Artifact(run.id), Artifact(run.created_at.strftime(DATETIME_FORMAT)), Artifact(str(self.scores.mean())), Markdown("This run...")])
        return rows
                
    
    @card(type="corise")
    @step
    def validate(self):
        from sklearn.model_selection import cross_val_score
        self.scores = cross_val_score(self.model, self.X, self.y, cv=5)
        current.card.append(Markdown("# Taxi Fare Prediction Results"))
        current.card.append(Table(self.gather_sibling_flow_run_results(), headers=["Pass/fail", "Run ID", "Created At", "R^2 score", "Stderr"]))
        self.next(self.end)

    @step
    def end(self):
        print("Success!")


if __name__ == "__main__":
    TaxiFarePrediction_Foo()

Overwriting ../flows/cloud/foo_taxi_fare_champion_1.py


In [44]:
! python ../flows/cloud/foo_taxi_fare_champion_1.py --environment=conda --production --branch champion argo-workflows create

Metaflow 2.9.1+ob(v1) executing TaxiFarePrediction_Foo for user:sandbox
Project: foo_taxi_fare, Branch: prod.champion
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
Deploying footaxifare.prod.champion.taxifarepredictionfoo to Argo Workflows...

The namespace of this production flow is
    production:mfprj-sh5wdhgelehxd3cq-0-hrge
To analyze results of this production flow add this line in your notebooks:
    namespace("production:mfprj-sh5wdhgelehxd3cq-0-hrge")
If you want to authorize other people to deploy new versions of this flow to Argo Workflows, they need to call
    argo-workflows create --authorize mfprj-sh5wdhgelehxd3cq-0-hrge
when deploying this flow to Argo Workflows for the first time.
See "Organizing Results" at https://docs.metaflow.org/ for more information about production tokens.

Bootstrapping conda environment...(this could take a few minutes)
Workflow footaxifare.prod.champion.taxifarepredictionfoo for flow TaxiFarePredictio

In [45]:
! python ../flows/cloud/foo_taxi_fare_champion_1.py --environment=conda --production --branch champion argo-workflows trigger

Metaflow 2.9.1+ob(v1) executing TaxiFarePrediction_Foo for user:sandbox
Project: foo_taxi_fare, Branch: prod.champion
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
Workflow footaxifare.prod.champion.taxifarepredictionfoo triggered on Argo Workflows (run-id argo-footaxifare.prod.champion.taxifarepredictionfoo-2ptl2).
See the run in the UI at https://ui-pw-1998985377.outerbounds.dev/TaxiFarePrediction_Foo/argo-footaxifare.prod.champion.taxifarepredictionfoo-2ptl2


## Task 2: Build the challenger
Develop a second model, by using the same `TaxiFarePrediction` architecture. Then, deploy the flow to Argo as the `--branch challenger`. 
<br>
<br>
Hint: Modify the `linear_model` step. 
<br>
Bonus: Write a paragraph summary of how you developed the second model and tested it before deploying the challenger flow. Let us know in Slack what you found challenging about the task? 

In [47]:
%%writefile ../flows/cloud/foo_taxi_fare_challenger_1.py

# Let's use a modified version of the event_triggered_linear_regression.py file from week 3 as the champion.
# Note that I encountered problems with reading the parquet url, and potentially saw some issues with caching - I had to explicitly state the parquet url and make sure
# to write the contents of the cell to a new file, using a different flow class name to make it work. Did not get to the root cause of the problem due to time limitations.

from metaflow import FlowSpec, step, card, conda_base, current, Parameter, Flow, trigger, project
from metaflow.cards import Markdown, Table, Image, Artifact


URL = 'https://outerbounds-datasets.s3.us-west-2.amazonaws.com/taxi/latest.parquet'
DATETIME_FORMAT = '%Y-%m-%d %H:%M:%S'

@project(name="foo_taxi_fare")
@trigger(events=['s3'])

@conda_base(libraries={'pandas': '1.4.2', 'pyarrow': '11.0.0', 'numpy': '1.21.2', 'scikit-learn': '1.1.2', 'xgboost': '1.7.4'})
class TaxiFarePrediction_Foo(FlowSpec):

    data_url = Parameter("data_url", default='https://outerbounds-datasets.s3.us-west-2.amazonaws.com/taxi/latest.parquet')

    def transform_features(self, df):

        # TODO: 
            # Try to complete tasks 2 and 3 with this function doing nothing like it currently is.
            # Understand what is happening.
            # Revisit task 1 and think about what might go in this function.
        obviously_bad_data_filters = [

            df.fare_amount > 0,         # fare_amount in US Dollars
            df.trip_distance <= 100,    # trip_distance in miles
            df.trip_distance > 0,
            df.passenger_count > 0,
            df.extra > 0,
            df.mta_tax > 0,
            df.tip_amount >= 0,
            df.tolls_amount >= 0,

            # TODO: add some logic to filter out what you decide is bad data!
            # TIP: Don't spend too much time on this step for this project though, it practice it is a never-ending process.

        ]

        for f in obviously_bad_data_filters:
            df = df[f]
        return df

    @step
    def start(self):

        import pandas as pd
        import io
        import requests
        from sklearn.model_selection import train_test_split

        self.df = self.transform_features(pd.read_parquet('https://outerbounds-datasets.s3.us-west-2.amazonaws.com/taxi/latest.parquet'))

        # NOTE: we are split into training and validation set in the validation step which uses cross_val_score.
        # This is a simple/naive way to do this, and is meant to keep this example simple, to focus learning on deploying Metaflow flows.
        # In practice, you want split time series data in more sophisticated ways and run backtests. 
        self.X = self.df["trip_distance"].values.reshape(-1, 1)
        self.y = self.df["total_amount"].values
        self.next(self.xgboost_model)

    @step
    def xgboost_model(self):
        from xgboost import XGBRegressor
        self.model = XGBRegressor()
        self.model_type = "xgboos"
        self.next(self.validate)

    def gather_sibling_flow_run_results(self):

        # storage to populate and feed to a Table in a Metaflow card
        rows = []

        # loop through runs of this flow 
        for run in Flow(self.__class__.__name__):
            if run.id != current.run_id:
                if run.successful:
                    icon = "✅" 
                    msg = "OK"
                    score = str(run.data.scores.mean())
                else:
                    icon = "❌"
                    msg = "Error"
                    score = "NA"
                    for step in run:
                        for task in step:
                            if not task.successful:
                                msg = task.stderr
                row = [Markdown(icon), Artifact(run.id), Artifact(run.created_at.strftime(DATETIME_FORMAT)), Artifact(score), Markdown(msg)]
                rows.append(row)
            else:
                rows.append([Markdown("✅"), Artifact(run.id), Artifact(run.created_at.strftime(DATETIME_FORMAT)), Artifact(str(self.scores.mean())), Markdown("This run...")])
        return rows
                
    
    @card(type="corise")
    @step
    def validate(self):
        from sklearn.model_selection import cross_val_score
        self.scores = cross_val_score(self.model, self.X, self.y, cv=5)
        current.card.append(Markdown("# Taxi Fare Prediction Results"))
        current.card.append(Table(self.gather_sibling_flow_run_results(), headers=["Pass/fail", "Run ID", "Created At", "R^2 score", "Stderr"]))
        self.next(self.end)

    @step
    def end(self):
        print("Success!")


if __name__ == "__main__":
    TaxiFarePrediction_Foo()

Overwriting ../flows/cloud/foo_taxi_fare_challenger_1.py


In [48]:
! python ../flows/cloud/foo_taxi_fare_champion_1.py --environment=conda --production --branch challenger argo-workflows create

Metaflow 2.9.1+ob(v1) executing TaxiFarePrediction_Foo for user:sandbox
Project: foo_taxi_fare, Branch: prod.challenger
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
Deploying footaxifare.prod.challenger.taxifarepredictionfoo to Argo Workflows...

The namespace of this production flow is
    production:mfprj-y22g5oex2j23upte-0-sezo
To analyze results of this production flow add this line in your notebooks:
    namespace("production:mfprj-y22g5oex2j23upte-0-sezo")
If you want to authorize other people to deploy new versions of this flow to Argo Workflows, they need to call
    argo-workflows create --authorize mfprj-y22g5oex2j23upte-0-sezo
when deploying this flow to Argo Workflows for the first time.
See "Organizing Results" at https://docs.metaflow.org/ for more information about production tokens.

Bootstrapping conda environment...(this could take a few minutes)
Workflow footaxifare.prod.challenger.taxifarepredictionfoo for flow TaxiFarePre

In [49]:
! python ../flows/cloud/foo_taxi_fare_champion_1.py --environment=conda --production --branch challenger argo-workflows trigger

Metaflow 2.9.1+ob(v1) executing TaxiFarePrediction_Foo for user:sandbox
Project: foo_taxi_fare, Branch: prod.challenger
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
Workflow footaxifare.prod.challenger.taxifarepredictionfoo triggered on Argo Workflows (run-id argo-footaxifare.prod.challenger.taxifarepredictionfoo-vsj7t).
See the run in the UI at https://ui-pw-1998985377.outerbounds.dev/TaxiFarePrediction_Foo/argo-footaxifare.prod.challenger.taxifarepredictionfoo-vsj7t


## Task 3: Analyze the results
Return to this notebook, and read in the results of the challenger and champion flow using the Metaflow Client API.
<br><br>

#### Questions
- Does your model perform better on the metrics you selected? 
- Think about your day job, how would you go about assessing whether to roll forward the production "champion" to your new model? 
    - What gives you confidence one model is better than another?
    - What kinds of information do you need to monitor to get buy-in from stakeholders that model A is preferable to model B?  

In [55]:
import numpy as np
from metaflow import Flow, namespace

# these values are unique to your deployment!
CHAMPION_MODEL_NAMESPACE = "production:mfprj-sh5wdhgelehxd3cq-0-hrge"
CHALLENGER_MODEL_NAMESPACE = "production:mfprj-y22g5oex2j23upte-0-sezo"

best_score = -1; winner = None; winner_namespace = None
for n in [CHAMPION_MODEL_NAMESPACE, CHALLENGER_MODEL_NAMESPACE]:
    namespace(n)
    run = Flow('TaxiFarePrediction_Foo').latest_successful_run
    acc_score = np.mean(run.data.scores)
    if acc_score > best_score:
        best_score = acc_score
        winner = run.data.model_type
        winner_namespace = n
print("The winner is the {} model, with accuracy of {}%. You can find the model in the flow deployed to the {} namespace.".format(winner, round(100*best_score, 2), winner_namespace))

The winner is the linear regressor model, with accuracy of 91.81%. You can find the model in the flow deployed to the production:mfprj-sh5wdhgelehxd3cq-0-hrge namespace.


I don't have satisfactory answers to the above questions: I spent too much time on mundane tasks instead of playing around with the core of the flows (getting the data from the parquet url, and coming to terms with what appeared to be caching issues).

In my day-job, we usually measure the quality of a model in terms of F1 score (with Precision being more important, traditionally). We also value high throughput and low latency (with focus being on the former). BUT it is not as easy as to put a better performing model into production... Some of our customers measure predictions over time, and are thus more interested in the delta differences at certain time points than the absolute scores. This means that they value consistency over quality. When speaking to their data scientists, they obviously want the best models, but it is their business department that "owns" the value chain on their end and they don't want changes they can't explain, or can't re-run for a long period of (back in) time.

## CONGRATULATIONS! 🎉✨🍾
If you made it this far, you have completed the Full Stack Machine Learning Corise course. 
We are so glad that you chose to learn with us, and hope to see you again in future courses. Stay tuned for more content and come join us in [Slack](http://slack.outerbounds.co/) to keep learning about Metaflow!